## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [24]:
import numpy as np
import pandas as pd
import pickle
import os

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import set_config

import functions_variables as fv
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA



from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
set_config(display="diagram")
X = pd.DataFrame()
source_folder_name = '../data'



In [2]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters


## Hyperparam Tuning

In [3]:
# perform tuning and cross validation here 
# using GridsearchCV/ RandomsearchCV (MVP)
# or your custom functions

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [4]:
# save your best model here
filename = 'best_model.pkl'
filepath = os.path.join('../data/models/', filename)
try:
    with open(filename, 'wb') as file:
        pickle.dump(best_model, file)
        print(f"file saved to: {filepath}")
except Exception as e:
    print(f"error saving model to:{filepath}: {e}")

error saving model to:../data/models/best_model.pkl: name 'best_model' is not defined


## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [5]:
class DataLoader(TransformerMixin):
    def __init__(self, source_folder_name):
        self.folder_name = source_folder_name

    def fit(self, X=None, y=None):  
        return self  

    def transform(self, X=None): 
        data = fv.load_data(self.folder_name)  
        return data

class DataCleaner(TransformerMixin):
    def __init__(self, num_type_to_drop):
        self.to_drop = num_type_to_drop

    def fit(self, X=None, y=None):
        return self

    def transform(self, X):  
        cleaned_data = fv.clean_data(X, self.to_drop)  
        return cleaned_data
    


In [6]:
class Scalar_numeric(TransformerMixin):
    def __init__(self):
        self.numeric_columns = None
        self.scalar = StandardScaler()
        
    def fit(self, X, y=None):
        self.numeric_columns = list(X.select_dtypes(include=['int', 'float']).columns)
        self.scalar.fit(X[self.numeric_columns])  
        return self
    
    def transform(self, X, y=None):
        X = X[self.numeric_columns].copy()
        X = self.scalar.transform(X)
        return X
    
class PCA_bool(TransformerMixin):
    def __init__(self, n_components):
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components)
        self.bool_columns = None

    def fit(self, X, y=None):
        self.bool_columns = X.select_dtypes(include=['bool']).columns
        self.pca.fit(X[self.bool_columns])
        return self
    
    def transform(self, X,y=None):
        X = X[self.bool_columns].copy()
        X = self.pca.transform(X)
        return X    

In [19]:
# Build pipeline here

pipe_load_clean = Pipeline([
    ('loader', DataLoader(source_folder_name)),
    ('cleaner', DataCleaner(num_type_to_drop=20))
    
])


union_pipe = FeatureUnion(transformer_list=[
    ('numeric', Scalar_numeric()),
    ('boolean', PCA_bool(n_components=10))
])

processing_pipe = Pipeline([
    ('scaling_union', union_pipe),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [8]:
cleaned_data = pipe_load_clean.fit_transform(X)
pipe_load_clean

Not a Json: .gitkeep
Not a Json: license.txt
Not a Json: models
Not a Json: processed
Not a Json: uscities.csv
Geocoding attempt 1 of 4...
Error geocoding Verona, Wisconsin: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Verona%2C+Wisconsin%2C+downtown&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding Concord, New Hampshire: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Concord%2C+New+Hampshire%2C+downtown&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding attempt 2 of 4...


In [12]:
pipe_load_clean

Pipeline(steps=[('loader', <__main__.DataLoader object at 0x000001C95F96D7F0>),
                ('cleaner',
                 <__main__.DataCleaner object at 0x000001C95F96DBE0>)])

In [27]:
processing_pipe

Pipeline(steps=[('scaling_union',
                 FeatureUnion(transformer_list=[('numeric',
                                                 <__main__.Scalar_numeric object at 0x000001C967455DC0>),
                                                ('boolean',
                                                 <__main__.PCA_bool object at 0x000001C967286730>)])),
                ('random_forest', RandomForestRegressor(random_state=42))])

In [25]:

X = cleaned_data.drop('sold_price', axis=1)
y = cleaned_data['sold_price']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8, random_state=42)


model = processing_pipe.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [26]:
print('rmse:', rmse,'\nmae:', mae, '\nr2:',r2)

rmse: 215585.83262187487 
mae: 120186.62409831975 
r2: 0.539954467849129


Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [30]:
model_dir = '../data/models'
filename = 'final_model.pkl'

filepath = os.path.join(model_dir, filename)

os.makedirs(model_dir, exist_ok=True) #makes directory if it does not exsist

try:
    with open(filepath, 'wb') as file:
        pickle.dump(model, file)
        print(f"Model saved to {filepath}")
except Exception as e:
    print(f"Error saving model: {e}")


Model saved to ../data/models\final_model.pkl
